# Importing the libraries

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time

# API key and ticker symbol

In [21]:
api_key = 'HDHS7RJSLYNN07J7'
ticker = 'TSLA'

# Importing the dataset

In [22]:
ts = TimeSeries(key = api_key, output_format = 'pandas')
df, meta_data = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full')
print(df.head())

                     1. open  2. high     3. low  4. close  5. volume
date                                                                 
2020-07-16 20:00:00  1487.99  1488.04  1487.9900   1488.04     1465.0
2020-07-16 19:59:00  1488.31  1488.41  1488.0399   1488.04     3659.0
2020-07-16 19:57:00  1488.95  1488.95  1488.9500   1488.95      486.0
2020-07-16 19:54:00  1489.03  1489.03  1488.0000   1488.00     2243.0
2020-07-16 19:53:00  1489.02  1489.02  1489.0200   1489.02      421.0


In [23]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3740 entries, 2020-07-16 20:00:00 to 2020-07-10 04:16:00
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   1. open    3740 non-null   float64
 1   2. high    3740 non-null   float64
 2   3. low     3740 non-null   float64
 3   4. close   3740 non-null   float64
 4   5. volume  3740 non-null   float64
dtypes: float64(5)
memory usage: 175.3 KB
None


# Data preprocessing

In [24]:
df.columns = ['o', 'h', 'l', 'c', 'vol'] #Renaming columns
color = []
rng = []
u_shadow = []
l_shadow = []
ub = []
lb = []
for ind in df.index:
    rng.append(df['h'][ind] - df['l'][ind]) #Calculating range
    
    if df['o'][ind] > df['c'][ind]:
        color.append('red') #open higher than close --> bearishness
    else:
        color.append('blue') #open is lower than or equal to close --> bulllishness

    if color[::-1] == 'red':  #Determining shadow length and bounds for red candles
        u_shadow.append(df['h'][ind] - df['o'][ind])
        l_shadow.append(df['c'][ind] - df['l'][ind])
        ub.append(df['o'][ind])
        lb.append(df['c'][ind])
    else: #Determining shadow length and bounds for blue candles
        u_shadow.append(df['h'][ind] - df['c'][ind])
        l_shadow.append(df['o'][ind] - df['l'][ind])
        ub.append(df['c'][ind])
        lb.append(df['o'][ind])
        
df['color'] = color
df['range'] = rng
df['u_shadow'] = u_shadow
df['l_shadow'] = l_shadow
df['u_bound'] = ub
df['l_bound'] = lb
print(df.head())

                           o        h          l        c     vol color  \
date                                                                      
2020-07-16 20:00:00  1487.99  1488.04  1487.9900  1488.04  1465.0  blue   
2020-07-16 19:59:00  1488.31  1488.41  1488.0399  1488.04  3659.0   red   
2020-07-16 19:57:00  1488.95  1488.95  1488.9500  1488.95   486.0  blue   
2020-07-16 19:54:00  1489.03  1489.03  1488.0000  1488.00  2243.0   red   
2020-07-16 19:53:00  1489.02  1489.02  1489.0200  1489.02   421.0  blue   

                      range  u_shadow  l_shadow  u_bound  l_bound  
date                                                               
2020-07-16 20:00:00  0.0500      0.00    0.0000  1488.04  1487.99  
2020-07-16 19:59:00  0.3701      0.37    0.2701  1488.04  1488.31  
2020-07-16 19:57:00  0.0000      0.00    0.0000  1488.95  1488.95  
2020-07-16 19:54:00  1.0300      1.03    1.0300  1488.00  1489.03  
2020-07-16 19:53:00  0.0000      0.00    0.0000  1489.02  1489.02 

# Single candlestick patterns

In [25]:
s_pattern = []
for ind in df.index:
    if (df['u_shadow'][ind]/df['u_bound'][ind] <= 0.003 and 
        df['l_shadow'][ind]/df['l_bound'][ind] <= 0.003):
        s_pattern.append('marubozu')
    elif df['u_shadow'][ind] == df['l_shadow'][ind]:
        if df['range'][ind] < 0.005:
            s_pattern.append('doji')
        else:
            s_pattern.append('spinning top')
    elif df['u_shadow'][ind] < 0.003 and df['l_shadow'][ind] > 2*df['range'][ind]:
        s_pattern.append('hammer')
    elif df['l_shadow'][ind] < 0.003 and df['u_shadow'][ind] > 2*df['range'][ind]:
        s_pattern.append('shooting star')
    else:
        s_pattern.append('others')
df['s_pattern'] = s_pattern
print(df.head())

                           o        h          l        c     vol color  \
date                                                                      
2020-07-16 20:00:00  1487.99  1488.04  1487.9900  1488.04  1465.0  blue   
2020-07-16 19:59:00  1488.31  1488.41  1488.0399  1488.04  3659.0   red   
2020-07-16 19:57:00  1488.95  1488.95  1488.9500  1488.95   486.0  blue   
2020-07-16 19:54:00  1489.03  1489.03  1488.0000  1488.00  2243.0   red   
2020-07-16 19:53:00  1489.02  1489.02  1489.0200  1489.02   421.0  blue   

                      range  u_shadow  l_shadow  u_bound  l_bound s_pattern  
date                                                                         
2020-07-16 20:00:00  0.0500      0.00    0.0000  1488.04  1487.99  marubozu  
2020-07-16 19:59:00  0.3701      0.37    0.2701  1488.04  1488.31  marubozu  
2020-07-16 19:57:00  0.0000      0.00    0.0000  1488.95  1488.95  marubozu  
2020-07-16 19:54:00  1.0300      1.03    1.0300  1488.00  1489.03  marubozu  
2020-0

## Analysing single candlestick pattern detection results

In [26]:
count = df['s_pattern'].value_counts()
print(count)

marubozu        3075
others           634
spinning top      31
Name: s_pattern, dtype: int64


In [18]:
for ind in df.index:
    if df['s_pattern'][ind] == 'shooting star':
        print(ind)

2020-07-14 16:05:00
2020-07-10 10:00:00
